In [1]:
!pip install silero torch torchaudio omegaconf -q

In [3]:
# =============================================================================
# ### Исправленный код с корректными именами спикеров ###
# =============================================================================
import torch
import torchaudio

# =============================================================================
# ### Актуальные параметры Silero TTS (2023.12) ###
# =============================================================================
# Доступные голоса (speaker) для русского языка:
# - 'aidar'       (мужской, 16kHz)
# - 'baya'        (женский, 16kHz)
# - 'kseniya'     (женский, 16kHz)
# - 'xenia'     (женский, 16kHz)
# - 'eugene'      (мужской, 16kHz)
# - 'random'       (женский, 16kHz)
#
# Доступные эмоции (emotion):
# - 'neutral'  - нейтральный
# - 'happy'    - радостный
# - 'sad'      - грустный
# - 'angry'    - злой



In [15]:
# =============================================================================
# ### Исправленный код с корректным распаковыванием значений ###
# =============================================================================
import os
import random
import torch
import torchaudio

def synthesize_speech(text, style):
    """
    Генерация речи с эмоциями и выбором голоса в зависимости от стиля.
    Актуально для версии Silero TTS 2024.07
    """
    style_config = {
        "1": {"speaker": "aidar", "emotion": "sad"},     # Космический ужас
        "2": {"speaker": "eugene", "emotion": "angry"},    # Гопнический стиль
        "3": {"speaker": "baya", "emotion": "surprise"},   # Экспериментальный
        "4": {"speaker": "kseniya", "emotion": "neutral"}  # По умолчанию
    }

    try:
        # Загрузка модели (возвращает только модель и пример текста)
        model, example_text = torch.hub.load(
            repo_or_dir='snakers4/silero-models',
            model='silero_tts',
            language='ru',
            speaker='v4_ru'  # Общая модель для русского языка
        )

        # Явное указание sample_rate для русских моделей
        sample_rate = 48000 #if "_16khz" in style_config[style]["speaker"] else 24000

        # Генерация аудио с обновленным API
        audio = model.apply_tts(
            text=text,  # Текст должен быть передан как список
            speaker=style_config[style]["speaker"],
            sample_rate=sample_rate
        )

        # Указание директории для аудиофайлов
        output_dir = '../audio'
        #os.makedirs(output_dir, exist_ok=True) #создание директории на случай, если она отсутствует

        # Генерация уникального имени файла (два варианта)
        # Вариант 1: Инкрементальное имя
        index = 1
        while os.path.exists(os.path.join(output_dir, f'output_{index}.wav')):
            index += 1
        output_file_increment = os.path.join(output_dir, f'output_{index}.wav')

        # Вариант 2: Случайное имя
        random_number = random.randint(1, 9999)
        output_file_random = os.path.join(output_dir, f'output_{random_number}.wav')

        # Выбор подхода: используем инкрементальный вариант для примера
        output_file = output_file_random

        # Сохранение аудио
        torchaudio.save(
            output_file,
            audio.unsqueeze(0),
            sample_rate
        )
        print(f"Успешно! Спикер: {output_file}, частота: {sample_rate} Гц")

    except Exception as e:
        print(f"Ошибка: {e}")
        print("Проверьте: 1) Имена спикеров 2) Версию PyTorch 3) Интернет-соединение")



In [16]:
# =============================================================================
# ### Пример использования ###
# =============================================================================

recipe = """Из представленных ингредиентов можно приготовить коктейль "Привет от Дориана Грей Гуса".

Ингредиенты:
- 15 мл ежевичного ликера;
- 15 мл персикового шнапса;
- 15 мл светлого рома;
- 15 мл лимонного сока.

Приготовление:
1. Соединить все ингредиенты в шейкере.
2. Добавить лед.
3. Хорошо взбалтывать.
4. Процедить в шот.

Приятного аппетита!""" # Ваш текст здесь

speaker = []
# Ввод с валидацией
while True:
    choice = input("Выберите стиль (1-4): ")
    if choice in {"1", "2", "3", "4"}:
        break
    print("Ошибка: допустимые значения 1-4")

synthesize_speech(recipe, choice)

Using cache found in /home/elijah/.cache/torch/hub/snakers4_silero-models_master


Успешно! Спикер: ../audio/output_8217.wav, частота: 48000 Гц
